# Importing the modules

In [50]:
import re
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences
import keras
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
import math
import nltk

# Reading the Dataset

In [51]:
data = pd.read_csv('IMDB Dataset.csv')
data

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
...,...,...
49995,I thought this movie did a down right good job...,positive
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,I am a Catholic taught in parochial elementary...,negative
49998,I'm going to have to disagree with the previou...,negative


# Data Processing
->Removing HTML Tags
->Removing urls
->Removing puncutations
->Stopwords removal

In [52]:
from string import punctuation
def remove_tags(string):
    removelist = ""
    result = re.sub('','',string)          #remove HTML tags
    result = re.sub('https://.*','',result)   #remove URLs
    result = ''.join([c for c in string if c not in punctuation])
    result = result.lower()
    return result
data['review']=data['review'].apply(lambda cw : remove_tags(cw))

In [53]:
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
data['review'] = data['review'].apply(lambda x: ' '.join([word for word in x.split() if word not in stop_words]))

[nltk_data] Downloading package stopwords to C:\Users\LOKESH
[nltk_data]     DANDAMUDI\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Tokenization

In [54]:
nltk.download('wordnet')
nltk.download('omw-1.4')
w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
lemmatizer = nltk.stem.WordNetLemmatizer()

[nltk_data] Downloading package wordnet to C:\Users\LOKESH
[nltk_data]     DANDAMUDI\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to C:\Users\LOKESH
[nltk_data]     DANDAMUDI\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


# Lemmatization

In [55]:
def lemmatize_text(text):
    st = ""
    for w in w_tokenizer.tokenize(text):
        st = st + lemmatizer.lemmatize(w) + " "
    return st
data['review'] = data.review.apply(lemmatize_text)
data

,review,sentiment
0,one reviewer mentioned watching 1 oz episode y...,positive
1,wonderful little production br br filming tech...,positive
2,thought wonderful way spend time hot summer we...,positive
3,basically there family little boy jake think t...,negative
4,petter matteis love time money visually stunni...,positive
...,...,...
49995,thought movie right good job wasnt creative or...,positive
49996,bad plot bad dialogue bad acting idiotic direc...,negative
49997,catholic taught parochial elementary school nu...,negative
49998,im going disagree previous comment side maltin...,negative


# Statistics about the Dataset

In [56]:
s = 0.0
for i in data['review']:
    word_list = i.split()
    s = s + len(word_list)
print("Average length of each review : ",s/data.shape[0])
pos = 0
for i in range(data.shape[0]):
    if data.iloc[i]['sentiment'] == 'positive':
        pos = pos + 1
neg = data.shape[0]-pos
print("Percentage of reviews with positive sentiment is "+str(pos/data.shape[0]*100)+"%")
print("Percentage of reviews with negative sentiment is "+str(neg/data.shape[0]*100)+"%")

Average length of each review :  122.83986
Percentage of reviews with positive sentiment is 50.0%
Percentage of reviews with negative sentiment is 50.0%


In [57]:
reviews = data['review'].values
labels = data['sentiment'].values

# Encoding and Train/Test split

In [58]:
encoder = LabelEncoder()
encoded_labels = encoder.fit_transform(labels)

In [59]:
X_train, X_test, y_train, y_test = train_test_split(reviews, encoded_labels, )

# Tokenization and Padding

In [61]:
# tokenize sentences
vocab_size = 3000
tokenizer = Tokenizer(num_words = vocab_size, oov_token='')
tokenizer.fit_on_texts(X_train)
word_index = tokenizer.word_index

In [62]:
# convert train dataset to sequence and pad sequences
train_sequences = tokenizer.texts_to_sequences(X_train)
train_padded = pad_sequences(train_sequences, padding='post', maxlen=200)

In [63]:
test_sequences = tokenizer.texts_to_sequences(X_test)
test_padded = pad_sequences(test_sequences, padding='post', maxlen=200)

# Building the Model

In [64]:
# model initialization
embedding_dim = 100
model = keras.Sequential([
    keras.layers.Embedding(vocab_size, embedding_dim, input_length=200),
    keras.layers.Bidirectional(keras.layers.LSTM(64)),
    keras.layers.Dense(24, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])

In [65]:
# compile model
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
# model summary
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 200, 100)          300000    
                                                                 
 bidirectional (Bidirectiona  (None, 128)              84480     
 l)                                                              
                                                                 
 dense (Dense)               (None, 24)                3096      
                                                                 
 dense_1 (Dense)             (None, 1)                 25        
                                                                 
Total params: 387,601
Trainable params: 387,601
Non-trainable params: 0
_________________________________________________________________


# Training the model

In [66]:
#Model training
num_epochs = 5
history = model.fit(train_padded, y_train,
                    epochs=num_epochs, verbose=1,
                    validation_split=0.1)

Epoch 1/5
1055/1055 [==============================] - 241s 224ms/step - loss: 0.4136 - accuracy: 0.8219 - val_loss: 0.3240 - val_accuracy: 0.8648
Epoch 2/5
1055/1055 [==============================] - 227s 216ms/step - loss: 0.2983 - accuracy: 0.8810 - val_loss: 0.3179 - val_accuracy: 0.8709
Epoch 3/5
1055/1055 [==============================] - 260s 247ms/step - loss: 0.2646 - accuracy: 0.8959 - val_loss: 0.3293 - val_accuracy: 0.8677
Epoch 4/5
1055/1055 [==============================] - 261s 248ms/step - loss: 0.2304 - accuracy: 0.9111 - val_loss: 0.3106 - val_accuracy: 0.8720
Epoch 5/5
1055/1055 [==============================] - 234s 222ms/step - loss: 0.2061 - accuracy: 0.9223 - val_loss: 0.3324 - val_accuracy: 0.8709


# Model Prediction

In [68]:
prediction = model.predict(test_padded)
# Get labels based on probability 1 if p>= 0.5 else 0
pred_labels = []
for i in prediction:
    if i >= 0.5:
        pred_labels.append(1)
    else:
        pred_labels.append(0)
print("Accuracy of prediction on test set : ", accuracy_score(y_test,pred_labels))

391/391 [==============================] - 26s 66ms/step
Accuracy of prediction on test set :  0.87232


# Experimentation

In [76]:
sentence = ["The movie was very touching and heart whelming",
            "I have never seen a terrible movie like this",
            "the movie is very bad and awful in doing it the wrong way"]

In [77]:
sequences = tokenizer.texts_to_sequences(sentence)
padded = pad_sequences(sequences, padding='post', maxlen=200)
prediction = model.predict(padded)
pred_labels = []
for i in prediction:
    if i >= 0.5:
        pred_labels.append(1)
    else:
        pred_labels.append(0)
for i in range(len(sentence)):
    print(sentence[i])
    if pred_labels[i] == 1:
        s = 'Positive'
    else:
        s = 'Negative'
    print("Predicted sentiment : ",s)

1/1 [==============================] - 0s 103ms/step
The movie was very touching and heart whelming
Predicted sentiment :  Positive
I have never seen a terrible movie like this
Predicted sentiment :  Negative
the movie is very bad and awful in doing it the wrong way
Predicted sentiment :  Negative
